In [9]:
import pandas as pd
import networkx as nx
import random
from scipy import special
from collections import defaultdict
import click
import os
from statsmodels.stats import multitest
import csv

In [12]:
"""
Constructs a NetworkX graph.

Input:
    - file : csv file

Output:
    - graph: NetworkX graph
"""
def construct_graph(file):
    df = pd.read_csv(file, sep = "\t", header=None, names=["Protein1", "interaction", "Protein2"])
    df_interactions = df.replace("in-complex-with", +1)
    df_interactions = df_interactions.replace("controls-expression-of", -1)
    df_interactions = df_interactions.replace("controls-state-change-of", -1)
    
    G = nx.DiGraph()
    
    for i in range(len(df_interactions)):
        prot1 = df_interactions.iloc[i,0]
        prot2 = df_interactions.iloc[i,2]
        interaction = df_interactions.iloc[i,1]
        G.add_node(prot1)
        G.add_node(prot2)
        G.add_edge(prot1, prot2)
        G[prot1][prot2]['relation'] = interaction
    
    for n, nbrs in G.adj.items():
        for nbr, eattr in nbrs.items():
            relation = eattr['relation']
        
    return G

In [13]:
path = "/Users/sophiakrix/Desktop/MechEnrichmentLab/NOTCH1_Intracellular.txt"
df = pd.read_csv(path, sep = "\t", header=None, names=["Protein1", "interaction", "Protein2"])

In [14]:
df.head()

,Protein1,interaction,Protein2
0,CCNC,in-complex-with,CDK8
1,CCNC,in-complex-with,CREBBP
2,CCNC,in-complex-with,EP300
3,CCNC,in-complex-with,KAT2A
4,CCNC,in-complex-with,KAT2B


In [15]:
df.columns

Index(['Protein1', 'interaction', 'Protein2'], dtype='object')

In [16]:
interactions_set = set(df.iloc[:,1])
interactions_set

{'controls-expression-of', 'controls-state-change-of', 'in-complex-with'}

In [17]:
interactions = df["interaction"].value_counts()
interactions

in-complex-with             234
controls-expression-of       72
controls-state-change-of     12
Name: interaction, dtype: int64

In [18]:
df_interactions = df.replace("in-complex-with", +1)
df_interactions = df_interactions.replace("controls-expression-of", -1)
df_interactions = df_interactions.replace("controls-state-change-of", -1)

In [19]:
df_interactions["interaction"].value_counts()

 1    234
-1     84
Name: interaction, dtype: int64

# NetworkX

In [20]:
G = nx.DiGraph()

In [21]:
len(df_interactions)

318

In [22]:
for i in range(len(df_interactions)):
    prot1 = df_interactions.iloc[i,0]
    prot2 = df_interactions.iloc[i,2]
    interaction = df_interactions.iloc[i,1]
    G.add_node(prot1)
    G.add_node(prot2)
    G.add_edge(prot1, prot2)
    G[prot1][prot2]['relation'] = interaction

In [23]:
for n, nbrs in G.adj.items():
    for nbr, eattr in nbrs.items():
        relation = eattr['relation']
        #print(n, nbr, relation)

In [24]:
plt.subplot(121)
nx.draw(G, with_labels=True, font_weight='bold')
#plt.subplot(122)
#nx.draw_shell(G, nlist=[range(5, 10), range(5)], with_labels=True, font_weight='bold')

NameError: name 'plt' is not defined

In [25]:
# Nr edges and nr of nodes 

print("Number of edges: ",G.number_of_edges(),"\nNumber of nodes: ",G.number_of_nodes())


Number of edges:  290 
Number of nodes:  47


# Algorithmic Development

In [26]:
"""
Randomly assigns labels of [-1,0,1] to nodes in a graph
Labels:
-1 : Downregulated
0 : No change
+1 : Upregulated

Input:
    - graph : the graph consisting of protein nodes 

Output:
    - prints list of nodes with associated attribute label
"""
def random_node_labels(graph):
    for node in graph.nodes():
        random_label = random.randint(-1,1)
        graph.nodes[node]['label'] = random_label
    print(graph.nodes.data())

In [27]:
random_node_labels(G)

[('CCNC', {'label': 0}), ('CDK8', {'label': 0}), ('CREBBP', {'label': -1}), ('EP300', {'label': 1}), ('KAT2A', {'label': 1}), ('KAT2B', {'label': 1}), ('MAML1', {'label': 0}), ('MAML2', {'label': -1}), ('MAML3', {'label': 0}), ('MAMLD1', {'label': 1}), ('NOTCH1', {'label': 1}), ('RBPJ', {'label': 0}), ('SNW1', {'label': 0}), ('HES1', {'label': 0}), ('HES5', {'label': -1}), ('HEY1', {'label': 0}), ('HEY2', {'label': -1}), ('HEYL', {'label': 0}), ('MYC', {'label': -1}), ('CUL1', {'label': -1}), ('FBXW7', {'label': 0}), ('RBX1', {'label': 0}), ('SKP1', {'label': -1}), ('HDAC10', {'label': -1}), ('NCOR1', {'label': -1}), ('NCOR2', {'label': 0}), ('TBL1XR1', {'label': 1}), ('TBL1X', {'label': 0}), ('HDAC11', {'label': -1}), ('HDAC1', {'label': -1}), ('HDAC2', {'label': 1}), ('HDAC3', {'label': 0}), ('HDAC4', {'label': -1}), ('HDAC5', {'label': 1}), ('HDAC6', {'label': 1}), ('HDAC7', {'label': 1}), ('HDAC8', {'label': 0}), ('HDAC9', {'label': -1}), ('TLE1', {'label': 1}), ('TLE2', {'label': 

In [28]:
"""
Caclulates the shortest path between two nodes.

Input:
    - graph : NetworkX graph
    - source : upstream source node


Output:
    - dictionary of shortest path nodes between source node and all other nodes in graph
"""
def shortest_path(graph, source):
    for target in graph.nodes():
        shortest_paths = nx.shortest_path(graph, source)
    return shortest_paths

In [29]:
shortest_path(G, "CCNC")

{'CCNC': ['CCNC'],
 'CDK8': ['CCNC', 'CDK8'],
 'CREBBP': ['CCNC', 'CREBBP'],
 'EP300': ['CCNC', 'EP300'],
 'KAT2A': ['CCNC', 'KAT2A'],
 'KAT2B': ['CCNC', 'KAT2B'],
 'MAML1': ['CCNC', 'MAML1'],
 'MAML2': ['CCNC', 'MAML2'],
 'MAML3': ['CCNC', 'MAML3'],
 'MAMLD1': ['CCNC', 'MAMLD1'],
 'NOTCH1': ['CCNC', 'NOTCH1'],
 'RBPJ': ['CCNC', 'RBPJ'],
 'SNW1': ['CCNC', 'SNW1'],
 'HES1': ['CCNC', 'CREBBP', 'HES1'],
 'HES5': ['CCNC', 'CREBBP', 'HES5'],
 'HEY1': ['CCNC', 'CREBBP', 'HEY1'],
 'HEY2': ['CCNC', 'CREBBP', 'HEY2'],
 'HEYL': ['CCNC', 'CREBBP', 'HEYL'],
 'MYC': ['CCNC', 'CREBBP', 'MYC'],
 'RBX1': ['CCNC', 'NOTCH1', 'RBX1'],
 'RPS27A': ['CCNC', 'NOTCH1', 'RPS27A'],
 'SKP1': ['CCNC', 'NOTCH1', 'SKP1'],
 'UBA52': ['CCNC', 'NOTCH1', 'UBA52'],
 'UBB': ['CCNC', 'NOTCH1', 'UBB'],
 'UBC': ['CCNC', 'NOTCH1', 'UBC'],
 'TBL1XR1': ['CCNC', 'RBPJ', 'TBL1XR1'],
 'TBL1X': ['CCNC', 'RBPJ', 'TBL1X'],
 'TLE1': ['CCNC', 'CREBBP', 'HES1', 'TLE1'],
 'TLE2': ['CCNC', 'CREBBP', 'HES1', 'TLE2'],
 'TLE3': ['CCNC', 'CR

In [78]:
"""
Check if node labels of source and target node are the same

Input:
    - graph: NetworkX graph
    - source: source upstream node

Output:
    - list of concordant and non-concordant nodes for the source node
"""


def count_concordance(graph, source):
    same_label = False

    nodes_dic = defaultdict(list)

    for target, path_nodes in shortest_path(graph, source).items():

        # check if node labels of source and target are the same
        if G.nodes[source]['label'] * G.nodes[target]['label'] is 1:
            same_label = True

        # multiply the edge labels
        #edge_label = [G[path_nodes[i]][path_nodes[i + 1]]['relation'] * G[path_nodes[i]][path_nodes[i + 1]]['relation']
                      #for i in range(len(path_nodes) - 1)]

        edge_label = 1
        for i in range(len(path_nodes)-1):
            temp_edge_label = G[path_nodes[i]][path_nodes[i+1]]['relation']
            edge_label *= temp_edge_label

        # concordant node
        if same_label == True and edge_label == +1:
            graph.nodes[target]['concordance'] = +1
            nodes_dic['concordant'].append(target)

        # non-concordant node
        if same_label == False and edge_label == -1:
            graph.nodes[target]['concordance'] = -1
            nodes_dic['non-concordant'].append(target)

        # no change node
        if G.nodes[source]['label'] == 0 and G.nodes[target]['label'] == 0:
            nodes_dic['no change'].append(target)

    return nodes_dic


In [79]:
count_concordance(G, "NOTCH1")

defaultdict(list,
            {'concordant': ['NOTCH1',
              'RBPJ',
              'RBX1',
              'RPS27A',
              'SKP1',
              'SNW1',
              'UBA52',
              'UBB',
              'UBC',
              'TBL1XR1',
              'TBL1X']})

In [80]:

"""
Returns a dictionary of the nodes of the graph with their according
     - shortest path nodes
     - concordant nodes
     - non-concordant nodes
     - no change nodes

Input:
    - graph

Output:
    - dictionary of nodes 
"""
def nodes_dictionary(graph):
    dic = {}
    for node in graph.nodes():
        dic[node] = {}

        # concordant, non-concordant and no change nodes
        dic[node] = count_concordance(graph, node)

        # shortest path nodes
        dic[node]['shortest_path'] = list(shortest_path(graph, node).keys())

    return dic


In [81]:
nodes_dictionary(G)['CCNC']

defaultdict(list,
            {'no change': ['CCNC',
              'CDK8',
              'MAML1',
              'MAML3',
              'RBPJ',
              'SNW1',
              'HES1',
              'HEY1',
              'HEYL',
              'RBX1',
              'RPS27A',
              'UBB',
              'TBL1X',
              'TLE3'],
             'shortest_path': ['CCNC',
              'CDK8',
              'CREBBP',
              'EP300',
              'KAT2A',
              'KAT2B',
              'MAML1',
              'MAML2',
              'MAML3',
              'MAMLD1',
              'NOTCH1',
              'RBPJ',
              'SNW1',
              'HES1',
              'HES5',
              'HEY1',
              'HEY2',
              'HEYL',
              'MYC',
              'RBX1',
              'RPS27A',
              'SKP1',
              'UBA52',
              'UBB',
              'UBC',
              'TBL1XR1',
              'TBL1X',
              'TLE1',
       

In [82]:
"""
Calculates the concordance for an upstream node with its downstream nodes
Probability of getting at least the number of state changes consistent
with the direction
Input:
    - graph
    - p : probability of achieving a result

Output:
    - dictionary of p-values and corrected p-values for concordance
"""
def calculate_concordance(graph):
    concordance_dic = {}

    #assert 0 <= p and p <= 1, "p must be within [0,1]"

    for hyp_node in graph.nodes():
        
        if hyp_node not in graph.nodes():
            raise ValueError(f"The node {hyp_node} is not in the graph.")
        
        # n is number of trials
        n = len(shortest_path(graph, hyp_node).keys())
        # k is number of successful predictions
        k = len(count_concordance(graph, hyp_node)['concordant'])

        bin_coeff = special.binom(n, k)
        concordance = bin_coeff * (0.5 ** k) * (1 - 0.5) ** (n - k)
        concordance_dic[hyp_node] = {}
        concordance_dic[hyp_node]['p_val'] = concordance

    # correction for multiple testing
    pval_list = [concordance_dic[hyp_node]['p_val'] for hyp_node in graph.nodes()]
    reject, pvals_corrected = multitest.multipletests(pval_list,alpha=0.05,method='bonferroni')[:2]
    corrected_concordance_dic = {}
    for node, pval in zip(graph.nodes(),pvals_corrected):
        concordance_dic[node]['p_val_corrected'] = pval

    return concordance_dic

In [83]:
calculate_concordance(G)


{'CCNC': {'p_val': 4.656612873077393e-10,
  'p_val_corrected': 2.1886080503463745e-08},
 'CDK8': {'p_val': 9.313225746154785e-10,
  'p_val_corrected': 4.377216100692749e-08},
 'CREBBP': {'p_val': 1.862645149230957e-09,
  'p_val_corrected': 8.754432201385498e-08},
 'EP300': {'p_val': 3.725290298461914e-09,
  'p_val_corrected': 1.7508864402770996e-07},
 'KAT2A': {'p_val': 0.06285522133111954, 'p_val_corrected': 1.0},
 'KAT2B': {'p_val': 0.06285522133111954, 'p_val_corrected': 1.0},
 'MAML1': {'p_val': 7.450580596923828e-09,
  'p_val_corrected': 3.501772880554199e-07},
 'MAML2': {'p_val': 0.14944598078727722, 'p_val_corrected': 1.0},
 'MAML3': {'p_val': 7.450580596923828e-09,
  'p_val_corrected': 3.501772880554199e-07},
 'MAMLD1': {'p_val': 0.14944598078727722, 'p_val_corrected': 1.0},
 'NOTCH1': {'p_val': 0.0971398875117302, 'p_val_corrected': 1.0},
 'RBPJ': {'p_val': 7.450580596923828e-09,
  'p_val_corrected': 3.501772880554199e-07},
 'SNW1': {'p_val': 7.450580596923828e-09,
  'p_val_co

In [84]:
"""
Writes the values for nodes, concordant_nodes, non_concordant_nodes, no_change_nodes, p_val, p_val_corrected to a csv file

Input:
- graph
- csv_output : path for output file 

Output:
- csv file
"""
def create_concordant_df(graph, csv_output):
    rows = []
    for node in graph.nodes():
        node_dict = {}
        node_dict['node'] = node
        node_dict['concordant_nodes'] = len(count_concordance(graph, node)['concordant'])
        node_dict['non_concordant_nodes'] = len(count_concordance(graph, node)['non-concordant'])
        node_dict['no_change'] = len(count_concordance(graph, node)['no change'])
        node_dict['p_val'] = calculate_concordance(graph)[node]['p_val']
        node_dict['p_val_corrected'] = calculate_concordance(graph)[node]['p_val_corrected']
        
        rows.append(node_dict)
        
    df_ = pd.DataFrame(rows)
    
    df_.to_csv(csv_output)
        

        
        

In [86]:
create_concordant_df(G, "/Users/sophiakrix/Desktop/Concordance_test.csv")